/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
## **0. Machine Learning Pipeline - Feature Engineering**
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

        En los siguientes Notebooks, repasaremos la implementación de cada uno de los pasos en la Canalización de Aprendizaje Automático (ML Pipeline).

        Discutiremos:

        1. Data Analysis
        2. **Feature Engineering**
        3. Feature Selection
        4. Model Training
        5. Obtaining Predictions / Scoring


        Usaremos el dataset de precios de casas disponible en  [Kaggle.com](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data). Consulte a continuación para más detalles.

        ===================================================================================================

        ## Predicción del Precio de Venta de Casas

        El objetivo del proyecto es construir un modelo de aprendizaje automático para predecir el precio de venta de viviendas basándose en diferentes variables explicativas que describen aspectos de las casas residenciales.


        ### ¿Por qué es esto importante?

        Predecir los precios de las casas es útil para identificar inversiones fructíferas o para determinar si el precio anunciado de una casa está sobreestimado o subestimado.

        ### ¿Cuál es el objetivo del modelo de aprendizaje automático?

        Nuestro objetivo es minimizar la diferencia entre el precio real y el precio estimado por nuestro modelo. Evaluaremos el rendimiento del modelo con el:

        1. mean squared error (MSE)
        2. root squared of the mean squared error (RMSE)
        3. r-squared (R2).


        ### ¿Cómo descargo el dataset?

        - Visita [Kaggle Website](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).

        - Recuerda hacer **log in**

        - Desplázate hasta el final de la página y haz clic en el enlace **'train.csv'**, , luego haz clic en el botón azul 'descargar' hacia la derecha de la pantalla para descargar el dataset.

        - Descarga el archivo llamado **'test.csv'** y guárdalo en el directorio junto con los notebooks.

        **Tenga en cuenta lo siguiente:**

        -  Necesita estar registrado en Kaggle para poder descargar los datasets.
        -  Necesita aceptar los términos y condiciones de la competencia para descargar el dataset
        -  Si guarda el archivo en el directorio junto con el cuaderno de Jupyter, entonces podrá ejecutar el código tal como está escrito aquí.

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
## **I. Carga de librerias**
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

##### **Paso 1: Reproducibility: Setting the seed**

With the aim to ensure reproducibility between runs of the same notebook, but also between the research and production environment, for each step that includes some element of randomness, it is extremely important that we **set the seed**.

In [1]:
# ============================ Importación de librerias ============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

#========================== Librerias de Regresión ==========================
#Librerias para mejores subconjuntos
import itertools
import time
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.api import OLS, add_constant
from sklearn.ensemble import IsolationForest#nos identifica las anomalias
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
#vif
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
#========================== Metricas ========================================
import sklearn.metrics as metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
#================== Transformación de datos =================================
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# para la yeo-johnson transformation
import scipy.stats as stats

# para mostrar todas las columnas del dataframe en el cuaderno
pd.pandas.set_option('display.max_columns', None)

#DESCARGAR PICKLE
import pickle

sns.set(style='whitegrid', context='notebook')
plt.style.use('default')

In [2]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# for the yeo-johnson transformation
import scipy.stats as stats

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

# to save the trained scaler class
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
## **II. Carga de Dataset de Datos**
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [3]:
# cargar dataset
data = pd.read_csv('./Input/data_mod_ttv.csv')

# filas y columnas del dataset
print(data.shape)

# visualise the dataset
data.head(3)

(1237, 37)


,FECHAENSAYO,Al2O3,C3A,C4AF,CaCO3,CaO,Cal Libre,Caliza,Cl,Consistencia Normal,Contracción en autoclave,Expansión en autoclave,Fe2O3,Fluidez,Fraguado final,Fraguado inicial,H2O Pasta,H2O mortero,K2O,MgO,Na2O,Polvo Filtro,Puzolana,Pérdida por ignición,RC 1 día,Reproceso,Residuo insoluble,Retenido M325,Retenido M450,SO3,SiO2,Superficie específica,Yeso,Álcalis Equivalentes,Tipo_Cemento,C3S,Periodo
0,2024-01-02,4.6,7.30,8.86,5.29,62.5,1.5,3.35,0.047,25.1,NaN,0.14,2.91,119.0,235.0,114.0,163.0,359.0,0.77,2.3,0.34,NaN,NaN,2.88,2250.0,NaN,0.84,1.2,6.8,2.90,20.1,4130.0,6.11,0.85,I,52.12,202401
1,2024-01-03,4.7,7.62,8.87,5.40,62.4,1.4,3.40,0.044,25.2,NaN,0.10,2.91,119.0,235.0,113.0,164.0,359.0,0.76,2.2,0.33,NaN,NaN,2.58,2240.0,NaN,0.87,1.2,6.8,2.82,19.8,4080.0,5.88,0.83,I,53.42,202401
2,2024-01-04,5.4,9.99,7.77,16.11,50.7,0.9,23.00,0.027,28.3,NaN,0.05,2.55,106.0,290.0,186.0,184.0,388.0,1.01,2.1,0.29,NaN,8.0,9.85,1300.0,0.0,NaN,1.6,7.6,2.47,21.9,6100.0,4.75,0.95,ICo,27.40,202401


In [4]:
data.shape

(1237, 37)

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
## **III. Tratamiento de Datos**
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

------------------------------------------------------------------------------------
#### **3.1. Eliminación de variables de baja variabilidad e Imputación de valores nulls**
------------------------------------------------------------------------------------

In [5]:
#----------------------------------------------------------------------------------------
# 2.1 Identificación de valores nulos
#----------------------------------------------------------------------------------------
dfsinnull = data.copy()
print(dfsinnull.isnull().sum())

#----------------------------------------------------------------------------------------
# 2.2 Definir un umbral del 30% para eliminar columnas con muchos valores nulos
#----------------------------------------------------------------------------------------
threshold = 0.99  # Umbral del 30%
dfsinnull = dfsinnull.loc[:, dfsinnull.isnull().mean() < threshold]

#----------------------------------------------------------------------------------------
# 2.3 Imputación de valores nulos según la variable
#----------------------------------------------------------------------------------------
for column in dfsinnull.columns:
    if dfsinnull[column].isnull().sum() > 0:
        if dfsinnull[column].dtype == 'object':  # Para variables categóricas
            dfsinnull[column].fillna(dfsinnull[column].mode()[0], inplace=True)  # Imputación con la moda
        else:
            # Prueba con diferentes métodos de imputación
            dfsinnull[column].fillna(0, inplace=True)  # Imputación con la media
            #dfsinnull[column].fillna(dfsinnull[column].mean(), inplace=True)  # Imputación con la media

FECHAENSAYO                    0
Al2O3                          1
C3A                            1
C4AF                           1
CaCO3                          1
CaO                            1
Cal Libre                      1
Caliza                         0
Cl                             1
Consistencia Normal            0
Contracción en autoclave    1236
Expansión en autoclave      1027
Fe2O3                          1
Fluidez                        0
Fraguado final                 0
Fraguado inicial               0
H2O Pasta                      0
H2O mortero                    0
K2O                            1
MgO                            1
Na2O                           1
Polvo Filtro                 993
Puzolana                     382
Pérdida por ignición           1
RC 1 día                       0
Reproceso                    382
Residuo insoluble            856
Retenido M325                  0
Retenido M450                  0
SO3                            1
SiO2      

C:\Users\enriq\AppData\Local\Temp\ipykernel_4788\2953486249.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfsinnull[column].fillna(0, inplace=True)  # Imputación con la media


In [6]:
columnas_eliminadas = set(data) - set(dfsinnull.columns)
print("Columnas eliminadas:", columnas_eliminadas)

Columnas eliminadas: {'Contracción en autoclave'}


------------------------------------------------------------------------------------
#### **2.3. Limpieza de Datos - Resistencia 1 día**
------------------------------------------------------------------------------------

*Se busca identificar si la distribución de la variable target "Valor Laboratorio" presenta una distribución diferente ante la totalidad de todas las variables*


In [7]:
df_prueba_ = dfsinnull

In [8]:
#====================================================================================================================
#=============  ELIMINAR RC 1 DÍA QUE NO CUMPLAN CON PROM +- 1.5 DESVIACIÓN=========================================
#====================================================================================================================
df0 = df_prueba_ #df_prueba_ICo_tt, aquí la data solo se realiza con la data de entrenamiento

columna_cemento = 'Tipo_Cemento'
columna_rcNdias = 'RC 1 día'

df0 = df0[df0[columna_rcNdias] > 0]
print('Antes de atípicos: ', df0.shape)

# Crear una función para eliminar outliers por "Tipo de Cemento"
def eliminar_outliers_por_categoria(df, columna_cemento, columna_rcNdias):
    df_sin_outliers = pd.DataFrame()
    for categoria in df[columna_cemento].unique():
        # Filtrar por categoría
        sub_df = df[df[columna_cemento] == categoria]
        # Calcular IQR
        Q1 = sub_df[columna_rcNdias].quantile(0.25)
        Q3 = sub_df[columna_rcNdias].quantile(0.75)
        IQR = Q3 - Q1 #sub_df.apply(stats.iqr) #Q3 - Q1
        # Definir límites
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        # Filtrar outliers y agregar al DataFrame resultante
        sub_df_sin_outliers = sub_df[(sub_df[columna_rcNdias] >= limite_inferior) & (sub_df[columna_rcNdias] <= limite_superior)]
        df_sin_outliers = pd.concat([df_sin_outliers, sub_df_sin_outliers])
    return df_sin_outliers

# Aplicar la función
df_train_test_val_so = eliminar_outliers_por_categoria(df0, columna_cemento, columna_rcNdias)

# Imprimir el tamaño después de eliminar los valores atípicos
print('Después de atípicos: ', df_train_test_val_so.shape)

Antes de atípicos:  (1237, 36)
Después de atípicos:  (1218, 36)


In [9]:
df_train_test_val_so['Tipo_Cemento'].unique()

array(['I', 'ICo', 'MS(MH)'], dtype=object)

------------------------------------------------------------------------------------
#### **3.2. Separación de data Train-Test-val por Tipo de Cemento**
------------------------------------------------------------------------------------

In [10]:
#====================================================================================================================
# Convertir la columna FECHAENSAYO a formato datetime
#====================================================================================================================
df_train_test_val_so['FECHAENSAYO'] = pd.to_datetime(df_train_test_val_so['FECHAENSAYO'], format='%Y-%m-%d')

#====================================================================================================================
# DEFINIR LA FECHA DE CORTE PARA VALIDACIÓN (4 días antes del día actual)
#====================================================================================================================
# Definir la fecha actual
fecha_actual = pd.Timestamp.now()

#====================================================================================================================
# Definir la fecha de corte como 4 días antes del día actual
#====================================================================================================================
fecha_corte = fecha_actual - pd.Timedelta(days=1)

# Definir la fecha de inicio de validación (se calcula restando 4 días a la fecha actual)
fecha_inicio_validacion = fecha_corte - pd.Timedelta(days=65)

#====================================================================================================================
# SEPARAR DATA DE ENTRENAMIENTOaaaa
#====================================================================================================================
# Datos de entrenamiento: registros anteriores al rango de validación
df_train_test_so = df_train_test_val_so.loc[df_train_test_val_so['FECHAENSAYO'] < fecha_inicio_validacion]

#====================================================================================================================
# SEPARAR DATA DE VALIDACIÓN
#====================================================================================================================
# Datos de validación: registros entre fecha_inicio_validacion y fecha_corte
df_val_so = df_train_test_val_so.loc[
    (df_train_test_val_so['FECHAENSAYO'] >= fecha_inicio_validacion) &
    (df_train_test_val_so['FECHAENSAYO'] <= fecha_corte)
]

# #====================================================================================================================
# # IDENTIFICACION DE FECHAS DE DATA
# #====================================================================================================================
from datetime import datetime
print("Data de Train, Test y Val: Del", pd.to_datetime(df_train_test_so['FECHAENSAYO'].min()),'al', df_train_test_so['FECHAENSAYO'].max(), df_train_test_so.shape) 

Data de Train, Test y Val: Del 2024-01-02 00:00:00 al 2025-01-06 00:00:00 (1043, 36)


In [11]:
# #====================================================================================================================
# # IDENTIFICACION DE FECHAS DE DATA
# #====================================================================================================================
from datetime import datetime
print("Data de Train, Test: Del", pd.to_datetime(df_train_test_so['FECHAENSAYO'].min()),'al', df_train_test_so['FECHAENSAYO'].max(), df_train_test_so.shape) 
# print("Data de Validación:   Del", df_val['FECHAENSAYO'].min(),'al', df_val['FECHAENSAYO'].max() , df_val.shape)

Data de Train, Test: Del 2024-01-02 00:00:00 al 2025-01-06 00:00:00 (1043, 36)


In [12]:
df_prueba_MS_tt = df_train_test_so[df_train_test_so['Tipo_Cemento'].isin(['MS(MH)'])]
df_prueba_ICo_tt = df_train_test_so[df_train_test_so['Tipo_Cemento'].isin(['ICo'])]
df_prueba_I_tt = df_train_test_so[df_train_test_so['Tipo_Cemento'].isin(['I'])]

df_prueba_MS_val = df_val_so[df_val_so['Tipo_Cemento'].isin(['MS(MH)'])]
df_prueba_ICo_val = df_val_so[df_val_so['Tipo_Cemento'].isin(['ICo'])]
df_prueba_I_val = df_val_so[df_val_so['Tipo_Cemento'].isin(['I'])]

#**Con esta data se harán las construcciones de los modelos a y las pruebas de Test a fin de validar que mantienen su performance con la historia registrada**
print(df_prueba_MS_tt.shape, df_prueba_ICo_tt.shape, df_prueba_I_tt.shape)
#**Con esta data se harán las pruebas de validaciones de los modelos a fin de validar que mantienen su performance pasado los meses**
print(df_prueba_MS_val.shape, df_prueba_ICo_val.shape, df_prueba_I_val.shape)

(264, 36) (466, 36) (313, 36)
(44, 36) (76, 36) (55, 36)


In [13]:
#====================================================================================================================
# Definir Tipo de Cemento y periodo de entrenamiento
#====================================================================================================================
# Se nombra un dataframe para el tipo de cemento a trabajar
df_data = df_prueba_ICo_tt.loc[df_prueba_ICo_tt['FECHAENSAYO'] >= '2024-01-01'] #df_train_test
df_data = df_data.reset_index(drop=True)
df_data.head(1)

,FECHAENSAYO,Al2O3,C3A,C4AF,CaCO3,CaO,Cal Libre,Caliza,Cl,Consistencia Normal,Expansión en autoclave,Fe2O3,Fluidez,Fraguado final,Fraguado inicial,H2O Pasta,H2O mortero,K2O,MgO,Na2O,Polvo Filtro,Puzolana,Pérdida por ignición,RC 1 día,Reproceso,Residuo insoluble,Retenido M325,Retenido M450,SO3,SiO2,Superficie específica,Yeso,Álcalis Equivalentes,Tipo_Cemento,C3S,Periodo
0,2024-01-04,5.4,9.99,7.77,16.11,50.7,0.9,23.0,0.027,28.3,0.05,2.55,106.0,290.0,186.0,184.0,388.0,1.01,2.1,0.29,0.0,8.0,9.85,1300.0,0.0,0.0,1.6,7.6,2.47,21.9,6100.0,4.75,0.95,ICo,27.4,202401


In [14]:
# //////////////////////////////////////////////////////////////////////////////
# 3. Tratamiento de datos
# //////////////////////////////////////////////////////////////////////////////

columns_var = df_data.select_dtypes(include=[np.number]).columns
# columns_var = columns_var.drop('Año')
columns_var = columns_var.drop('Periodo')
# columns_var = columns_var.drop('RC 3 días')
# columns_var = columns_var.drop('RC 7 días')
# columns_var = columns_var.drop('RC 28 días')

#dato_new_numeric = pd.DataFrame(df_data.iloc[:,4:])
dato_new_numeric = pd.DataFrame(df_data[columns_var])

import pandas as pd
import numpy as np

def clean_data_v6(df, zero_threshold=0.7, min_nonzero_values=5):
    """
    Función que corrige outliers usando cuartiles, pero:
    - No imputa ceros históricos.
    - No imputa columnas con demasiados ceros (>zero_threshold).
    - No imputa columnas con pocos valores distintos de cero (min_nonzero_values).
    - Solo reemplaza valores outliers distintos de cero usando cuartiles.
    """
    columnas_numeric = df.select_dtypes(include=[np.number]).columns

    for colum in columnas_numeric:
        # Contar cantidad de valores cero y valores distintos de cero
        num_zeros = (df[colum] == 0).sum()
        num_nonzero_values = (df[colum] != 0).sum()
        total_values = len(df[colum])

        # Evaluar si la columna tiene un alto porcentaje de ceros
        zero_percentage = num_zeros / total_values

        # Evaluar si la columna tiene pocos valores distintos de cero
        if zero_percentage > zero_threshold:
            print(f"⚠️ La columna '{colum}' tiene un {zero_percentage:.1%} de ceros. No se aplicará imputación.")
            continue

        if num_nonzero_values < min_nonzero_values:
            print(f"⚠️ La columna '{colum}' tiene solo {num_nonzero_values} valores distintos de cero. No se aplicará imputación.")
            continue

        # Calcular cuartiles para detectar outliers
        Q1 = df[colum].quantile(0.25)
        Q3 = df[colum].quantile(0.75)
        IQR = Q3 - Q1  # Rango intercuartílico

        # Calcular límites inferior y superior
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR

        # Aplicar imputación SOLO a valores fuera de los límites y que no sean ceros históricos
        df[colum] = df[colum].apply(
            lambda x: Q1 if (x < lower_limit and x != 0) else 
                      (Q3 if (x > upper_limit and x != 0) else x)
        )

    return df

# Aplicar la función a los datos numéricos
dato_new_numeric = pd.DataFrame(df_data.iloc[:, 4:])
dato_new_numeric = clean_data_v6(dato_new_numeric)

# Ver resultados estadísticos después de la limpieza
print(dato_new_numeric.describe())
dato_new_numeric['Periodo'] = df_data['Periodo']

⚠️ La columna 'Expansión en autoclave' tiene un 79.6% de ceros. No se aplicará imputación.
⚠️ La columna 'Polvo Filtro' tiene un 85.2% de ceros. No se aplicará imputación.
⚠️ La columna 'Reproceso' tiene un 100.0% de ceros. No se aplicará imputación.
⚠️ La columna 'Residuo insoluble' tiene un 100.0% de ceros. No se aplicará imputación.
            CaCO3         CaO   Cal Libre      Caliza          Cl  \
count  466.000000  466.000000  466.000000  466.000000  466.000000   
mean    17.923686   54.990558    0.745279   27.393326    0.021929   
std      1.118778    2.738337    0.091753    2.303841    0.004742   
min     15.350000   50.400000    0.600000   22.500000    0.000000   
25%     17.137500   52.500000    0.700000   26.500000    0.020000   
50%     17.940000   54.150000    0.700000   27.500000    0.022000   
75%     18.675625   57.775000    0.800000   30.150000    0.025000   
max     20.760000   61.800000    0.900000   30.650000    0.032000   

       Consistencia Normal  Expansión en

------------------------------------------------------------------------------------
#### **3.4. Eliminación de Variables con Multicolinealidad (VIF)**
------------------------------------------------------------------------------------

In [15]:
df_data = dato_new_numeric
columnas_numeric = df_data.select_dtypes(include=['float64','int64']).columns
#columnas_numeric = columnas_numeric.drop('Año')
columnas_numeric = columnas_numeric.drop('Periodo')
#columnas_numeric = columnas_numeric.drop('FECHAENSAYO')
columnas_numeric = columnas_numeric.drop(columna_rcNdias) # 'RC 1 día'
# columnas_numeric = columnas_numeric.drop('RC 3 días') # 'RC 28 días'
# columnas_numeric = columnas_numeric.drop('RC 7 días') # 'RC 28 días'
# columnas_numeric = columnas_numeric.drop('RC 28 días') # 'RC 28 días'

In [16]:
#====================================================================================================================
# METODO DE VIF
#====================================================================================================================

import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

def compute_vif(df):
    """
    Calcula el VIF para un DataFrame numérico y devuelve un DataFrame ordenado por VIF descendente.
    """
    df = df.copy()  # Evita modificaciones sobre la referencia original
    
    # Filtrar solo columnas numéricas
    df = df.select_dtypes(include=[np.number])
    
    # Eliminar columnas constantes (varianza 0)
    df = df.loc[:, df.var() > 0]

    # Reemplazar valores NaN e Inf antes del cálculo del VIF
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)

    # Agregar intercepto para el cálculo del VIF
    df["intercept"] = 1

    vif_data = pd.DataFrame()
    vif_data["Variable"] = df.columns
    vif_data["VIF"] = [
        variance_inflation_factor(df.values, i) if df.iloc[:, i].var() > 0 else np.nan
        for i in range(df.shape[1])
    ]

    # Eliminar la columna intercept
    vif_data = vif_data[vif_data["Variable"] != "intercept"]

    return vif_data.sort_values("VIF", ascending=False)

#====================================================================================================================
# EJECUCIÓN DEL MÉTODO VIF
#====================================================================================================================

# Cargar los datos correctamente
df = df_data[columnas_numeric].copy()

# Lista para almacenar variables eliminadas
variables_eliminadas = []

# Calcular el VIF inicial
vif_df = compute_vif(df)

# Verificar si hay valores antes de iterar
if not vif_df.empty:
    vif_max = vif_df["VIF"].max()  # Obtener el VIF más alto
else:
    vif_max = 0

# Iterativamente eliminar variables con VIF >= 5
while vif_max >= 5 and not vif_df.empty:
    # Eliminar la variable con mayor VIF
    var_to_remove = vif_df.iloc[0, 0]
    
    if var_to_remove in df.columns:
        df.drop(columns=[var_to_remove], inplace=True)
        variables_eliminadas.append(var_to_remove)  # Registrar la variable eliminada

    # Recalcular el VIF con las variables restantes
    vif_df = compute_vif(df)

    # Verificar si hay valores antes de continuar iterando
    if not vif_df.empty:
        vif_max = vif_df["VIF"].max()  # Obtener el nuevo VIF más alto
    else:
        break  # Terminar si no hay más variables

# Imprimir el resultado final
print("Variables eliminadas en el proceso:")
print(variables_eliminadas)
print("\nVIF Final después de la eliminación:")
print(vif_df)

Variables eliminadas en el proceso:
['H2O Pasta', 'SiO2', 'Álcalis Equivalentes', 'H2O mortero', 'CaO', 'Puzolana', 'Consistencia Normal', 'Caliza', 'Fraguado inicial', 'K2O', 'Pérdida por ignición']

VIF Final después de la eliminación:
                  Variable       VIF
15                     C3S  3.695016
11           Retenido M450  3.528676
6           Fraguado final  3.183921
13   Superficie específica  3.047646
0                    CaCO3  2.937700
10           Retenido M325  2.867504
14                    Yeso  2.410683
3   Expansión en autoclave  2.207705
4                    Fe2O3  2.203688
8                     Na2O  2.033584
9             Polvo Filtro  2.033396
7                      MgO  1.845677
12                     SO3  1.269737
2                       Cl  1.224732
1                Cal Libre  1.124623
5                  Fluidez  1.098962


In [17]:
#====================================================================================================================
# SEPARACIÓN Y LISTADO DE CAMPOS 
#====================================================================================================================

variables_vif = list(vif_df['Variable'])
variables_vif.append(columna_rcNdias) # Columna para las resistencias
variables_vif

['C3S',
 'Retenido M450',
 'Fraguado final',
 'Superficie específica',
 'CaCO3',
 'Retenido M325',
 'Yeso',
 'Expansión en autoclave',
 'Fe2O3',
 'Na2O',
 'Polvo Filtro',
 'MgO',
 'SO3',
 'Cl',
 'Cal Libre',
 'Fluidez',
 'RC 1 día']

In [18]:
#====================================================================================================================
# DELIMITACION DE LOS CAMPOS Y REGISTROS QUE SE QUEDAN EN EL PROCEOSO
#====================================================================================================================
data_pre_mod = pd.DataFrame(dato_new_numeric[variables_vif]) #    [list(vif_a2['Variable'])])
data_pre_mod[columna_rcNdias] = dato_new_numeric[columna_rcNdias] # 'RC 28 días' #'RC 28 días'
data_pre_mod.head(3)

,C3S,Retenido M450,Fraguado final,Superficie específica,CaCO3,Retenido M325,Yeso,Expansión en autoclave,Fe2O3,Na2O,Polvo Filtro,MgO,SO3,Cl,Cal Libre,Fluidez,RC 1 día
0,27.40,7.6,290.0,6100.0,16.11,1.6,4.75,0.05,2.55,0.29,0.0,2.1,2.47,0.027,0.9,106.0,1300.0
1,31.76,10.8,275.0,6080.0,15.44,3.2,4.91,0.05,2.57,0.30,0.0,2.1,2.33,0.025,0.8,106.0,1450.0
2,31.01,8.8,265.0,5890.0,15.91,2.0,4.56,0.06,2.48,0.30,0.0,2.1,2.34,0.025,0.8,107.0,1470.0


In [19]:
variances = round(data_pre_mod.var(),2)
print(variances)

C3S                          13.49
Retenido M450                 1.68
Fraguado final              212.77
Superficie específica     69998.66
CaCO3                         1.25
Retenido M325                 0.28
Yeso                          0.05
Expansión en autoclave        0.00
Fe2O3                         0.01
Na2O                          0.00
Polvo Filtro                  0.12
MgO                           0.01
SO3                           0.01
Cl                            0.00
Cal Libre                     0.01
Fluidez                       2.69
RC 1 día                  48193.24
dtype: float64


In [20]:
from sklearn.feature_selection import VarianceThreshold

# Eliminar variables con baja varianza
threshold = 0.0001  # Umbral de varianza mínima
selector = VarianceThreshold(threshold=threshold)

# # Aplicar la selección de características
X_selected_var = selector.fit_transform(data_pre_mod)

# Obtener los nombres de las variables seleccionadas
selected_columns = data_pre_mod.columns[selector.get_support()]
#print(f"Variables seleccionadas después de la eliminación por baja varianza: {selected_columns}")
eliminated_columns = data_pre_mod.columns[~selector.get_support()]
print("Columnas Eliminadas:", eliminated_columns)
print("Columnas Seleccionadas:", selected_columns)

Columnas Eliminadas: Index(['Cl'], dtype='object')
Columnas Seleccionadas: Index(['C3S', 'Retenido M450', 'Fraguado final', 'Superficie específica',
       'CaCO3', 'Retenido M325', 'Yeso', 'Expansión en autoclave', 'Fe2O3',
       'Na2O', 'Polvo Filtro', 'MgO', 'SO3', 'Cal Libre', 'Fluidez',
       'RC 1 día'],
      dtype='object')


In [21]:
var_data_pre_mod = list(selected_columns)
data_pre_mod_vth = data_pre_mod[var_data_pre_mod]
data_pre_mod_vth.head(2)

,C3S,Retenido M450,Fraguado final,Superficie específica,CaCO3,Retenido M325,Yeso,Expansión en autoclave,Fe2O3,Na2O,Polvo Filtro,MgO,SO3,Cal Libre,Fluidez,RC 1 día
0,27.40,7.6,290.0,6100.0,16.11,1.6,4.75,0.05,2.55,0.29,0.0,2.1,2.47,0.9,106.0,1300.0
1,31.76,10.8,275.0,6080.0,15.44,3.2,4.91,0.05,2.57,0.30,0.0,2.1,2.33,0.8,106.0,1450.0


------------------------------------------------------------------------------------
#### **3.5. Separación de data de Train y Test**
------------------------------------------------------------------------------------

In [22]:
numeric_cols = data_pre_mod_vth.select_dtypes(include=['float64','int']).columns
numeric_cols = numeric_cols.drop(columna_rcNdias) #'RC 1 día'

# variables ya resumidas con las finales de VIF
X = data_pre_mod_vth[numeric_cols]
y = data_pre_mod_vth[columna_rcNdias]

# splitting X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state= 1 ) #stratify=y,

In [61]:
# # Let's separate into train and test set
# # Remember to set the seed (random_state for this sklearn function)

# X_train, X_test, y_train, y_test = train_test_split(
#     data.drop(['Id', 'SalePrice'], axis=1), # predictive variables
#     data['SalePrice'], # target
#     test_size=0.1, # portion of dataset to allocate to test set
#     random_state=0, # we are setting the seed here
# )

# X_train.shape, X_test.shape

------------------------------------------------------------------------------------
#### **3.5. Paso 3: Feature Engineering (Ingeniería de Características)**
------------------------------------------------------------------------------------

        En las siguientes celdas, procesaremos las variables del dataset de Precios de Casas para abordar:

        1. Missing values (NaN - Valores Faltantes)
        2. Temporal variables (Variables Temporales)
        3. Non-Gaussian distributed variables (Variables con distribución no gaussiana)
        4. Categorical variables: remove rare labels (Variables categóricas: eliminar etiquetas raras)
        5. Categorical variables: convert strings to numbers (Variables categóricas: convertir cadenas a números)
        5. Put the variables in a similar scale (Poner las variables en una escala similar)

##### Target

Aplicamos el logaritmo

In [23]:
y_train = np.log(y_train)
y_test = np.log(y_test)

#### **Numerical variable transformation**

##### **Logarithmic transformation**

In the previous notebook, we observed that the numerical variables are not normally distributed.

We will transform with the logarightm the positive numerical variables in order to get a more Gaussian-like distribution.

In [ ]:
# for var in [numeric_cols]:
#     X_train[var] = np.log(X_train[var])
#     X_test[var] = np.log(X_test[var])

c:\Users\enriq\anaconda3\envs\DataPat-MLOps-11\lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


#### **Yeo-Johnson transformation**

We will apply the Yeo-Johnson transformation to LotArea.

In [105]:
# # the yeo-johnson transformation learns the best exponent to transform the variable
# # it needs to learn it from the train set:
# X_train['LotArea'], param = stats.yeojohnson(X_train['LotArea'])

# # and then apply the transformation to the test set with the same
# # parameter: see who this time we pass param as argument to the
# # yeo-johnson
# X_test['LotArea'] = stats.yeojohnson(X_test['LotArea'], lmbda=param)

# print(param)

In [ ]:
# # check absence of na in the train set
# [var for var in X_train.columns if X_train[var].isnull().sum() > 0]

[]

In [ ]:
# # check absence of na in the test set
# [var for var in X_train.columns if X_test[var].isnull().sum() > 0]

[]

#### **Binarize skewed variables (Binarizar variables sesgadas)**

Había algunas variables muy sesgadas, las transformaremos en variables binarias.

In [ ]:
# skewed = [
#     'BsmtFinSF2', 'LowQualFinSF', 'EnclosedPorch',
#     '3SsnPorch', 'ScreenPorch', 'MiscVal'
# ]

# for var in skewed:

#     # map the variable values into 0 and 1
#     X_train[var] = np.where(X_train[var]==0, 0, 1)
#     X_test[var] = np.where(X_test[var]==0, 0, 1)

------------------------------------------------------------------------------------
#### **3.x. Variables Categoricas (NO Utilizadas)**
------------------------------------------------------------------------------------

###### Variables categóricas

###### Aplicamos mapeos

Estas son variables cuyos valores tienen un orden asignado, relacionado con la calidad. Para más información, consulta el sitio web de Kaggle.

##### Encoding of categorical variables

A continuación, necesitamos transformar las cadenas de las variables categóricas en números.

Lo haremos para capturar la relación monótona entre la etiqueta y el objetivo.

In [37]:
# esta función asignará valores discretos a las cadenas de las variables,
# de modo que el valor más pequeño corresponda a la categoría que muestra el menor
# precio medio de venta de casas

def replace_categories(train, test, y_train, var, target):

    tmp = pd.concat([X_train, y_train], axis=1)

    # order the categories in a variable from that with the lowest
    # house sale price, to that with the highest
    ordered_labels = tmp.groupby([var])[target].mean().sort_values().index

    # create a dictionary of ordered categories to integer values
    ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

    print(var, ordinal_label)
    print()

    # use the dictionary to replace the categorical strings by integers
    train[var] = train[var].map(ordinal_label)
    test[var] = test[var].map(ordinal_label)

###### Verificamos ¿Hay valores nulos (na) en el conjunto de Entrenamiento? - NO

###### Verificamos ¿Hay valores nulos (na) en el conjunto de Prueba? - NO

La relación monótona es particularmente clara para las variables MSZoning y Neighborhood. Observa cómo, cuanto mayor es el número entero que ahora representa la categoría, mayor es el precio medio de venta de casas.

(recuerda que el objetivo está transformado logarítmicamente, por eso las diferencias parecen tan pequeñas).

------------------------------------------------------------------------------------
#### **3.6. Feature Scaling**
------------------------------------------------------------------------------------

Para su uso en modelos lineales, las características deben ser escaladas. Escalaremos las características a los valores mínimos y máximos:

In [31]:
if isinstance(X_train, pd.DataFrame):
    print("X_train es un DataFrame de pandas")
else:
    print("X_train no es un DataFrame de pandas")

X_train es un DataFrame de pandas


In [32]:
print(len(X_train))
print(X_train.shape)

372
(372, 15)


In [33]:
#DESCARGAR PICKLE
from sklearn.preprocessing import MinMaxScaler
import pickle

# Estandarizar los valores
scaler_x = MinMaxScaler().fit(X_train.select_dtypes(include=['float64','int']))
# Guardar el modelo y el escalador en un archivo .pkl
with open('MinMaxScale_1d_Ico.pkl', 'wb') as file:
    pickle.dump((scaler_x), file)    
    
# Cargar el archivo Pickle
with open('MinMaxScale_1d_Ico.pkl', 'rb') as file:
    MinMaxScale_1d_Ico = pickle.load(file)

In [ ]:
# REVISAR EL NUMERO DE VARIABLES EN NUMERIC COLS Y DATA PRE MOD
from sklearn import preprocessing
from sklearn.preprocessing import scale

Xstandar = MinMaxScale_1d_Ico.transform(X_train.select_dtypes(include=['float64','int']))
dato_num_s = pd.DataFrame(data=Xstandar, columns = numeric_cols)
# dato_num_s[columna_rcNdias] = data_pre_mod_vth[columna_rcNdias]
X_train = dato_num_s
X_train.head(3)

,C3S,Retenido M450,Fraguado final,Superficie específica,CaCO3,Retenido M325,Yeso,Expansión en autoclave,Fe2O3,Na2O,Polvo Filtro,MgO,SO3,Cal Libre,Fluidez
0,0.475299,0.722222,0.500000,0.465517,0.236599,0.5,0.538462,0.0,0.480769,0.727273,0.0,0.666667,0.500000,0.666667,0.000000
1,0.155486,0.666667,0.357143,0.793103,0.683919,1.0,0.250000,0.0,0.173077,0.454545,0.0,0.333333,0.452381,0.000000,0.285714
2,0.165367,0.611111,0.785714,0.655172,0.451017,0.5,0.826923,0.0,0.519231,0.636364,0.0,0.333333,0.571429,0.666667,0.714286


In [ ]:
Xstandar_t = MinMaxScale_1d_Ico.transform(X_test.select_dtypes(include=['float64','int']))
dato_num_t = pd.DataFrame(data=Xstandar_t, columns = numeric_cols)
X_test = dato_num_t
X_test.head(3)

,C3S,Retenido M450,Fraguado final,Superficie específica,CaCO3,Retenido M325,Yeso,Expansión en autoclave,Fe2O3,Na2O,Polvo Filtro,MgO,SO3,Cal Libre,Fluidez
0,0.491420,0.833333,0.071429,0.043103,0.683919,0.666667,0.711538,0.0,0.076923,0.272727,0.0,0.666667,0.619048,0.333333,0.571429
1,0.682267,0.388889,0.285714,0.413793,0.691312,0.333333,0.211538,0.0,0.134615,0.272727,1.0,0.666667,0.452381,0.000000,0.285714
2,0.312533,0.500000,0.285714,0.681034,0.545287,0.000000,0.548077,0.0,0.269231,0.818182,0.0,0.333333,0.309524,0.333333,0.000000


In [111]:
# # Creamos el Escalador (Scaler)
# scaler = MinMaxScaler()

# # Ajustamos el Scaler para el Conjunto de Entrenamiento
# scaler.fit(X_train)

# # Transformamos el conjunto de entrenamiento y el conjunto de prueba

# # sklearn devuelve arrays de numpy, así que envolvemos el
# # array con un dataframe de pandas

# X_train = pd.DataFrame(
#     scaler.transform(X_train),
#     columns=X_train.columns
# )

# X_test = pd.DataFrame(
#     scaler.transform(X_test),
#     columns=X_train.columns
# )

In [40]:
# Guardemos ahora los conjuntos de entrenamiento y prueba para el próximo Notebook!

X_train.to_csv('./Output/xtrain.csv', index=False)
X_test.to_csv('./Output/xtest.csv', index=False)

y_train.to_csv('./Output/ytrain.csv', index=False)
y_test.to_csv('./Output/ytest.csv', index=False)

In [ ]:
# # Ahora guardamos el Scaler

# joblib.dump(scaler, './Output/minmax_scaler.joblib')

['./Output/minmax_scaler.joblib']